In [ ]:
import numpy as np
import pandas as pd

import nltk
import torch
from transformers import T5Tokenizer, AutoModelForSeq2SeqLM
from peft import PeftConfig, PeftModel
from t5.dataset import load_spider_datasets, DatasetIterator
from t5.model import BaseModel, set_train_arguments
from t5.prompt_tuning import *

In [ ]:
nltk.download('punkt')

In [ ]:
!huggingface-cli login --token 

In [ ]:
print(torch.cuda.is_available())

## Model Parameters

In [ ]:
model_name = "t5-base"
technique = "soft_prompt_tune"
version = 1
checkpoint = 2190

folder_name = f"{model_name}_{technique}_{version}"
train_path = f"results/{folder_name}"
model_path = train_path + f'/{folder_name}'
last_check_point = train_path + f'/checkpoint-{checkpoint}'

# model name on hugging face
hug_model_name = "RoxyRong/t5_base_finetuned_15"

print("train_path:", train_path)
print("model_path:", model_path)

In [ ]:
# soft prompt tuning param
num_of_tokens = 100
base_model_name = "RoxyRong/t5_base_finetuned"
init_text = init_random_vocab_from_tokenizer()
size = 32128

tokenizer = T5Tokenizer.from_pretrained(model_name)
peft_config = set_peft_config(init_text=init_text, num_of_tokens=num_of_tokens)
model = AutoModelForSeq2SeqLM.from_pretrained(base_model_name)
model = PeftModel.from_pretrained(model, peft_config)
model = model.to("cuda")

## Dataset

In [ ]:
train_spider, others_spider, dev_spider = load_spider_datasets()
train_spider = train_spider.iloc[np.random.permutation(train_spider.index)].reset_index(drop=True)
others_spider = train_spider.iloc[np.random.permutation(others_spider.index)].reset_index(drop=True)

In [ ]:
max_load_at_once = 100

train_data_iterator = DatasetIterator(
    df=train_spider,
    tokenizer=tokenizer,
    max_load_at_once=max_load_at_once,
)

valid_data_iterator = DatasetIterator(
    df=others_spider,
    tokenizer=tokenizer,
    max_load_at_once=max_load_at_once,
)

## Model Training

In [ ]:
batch_size = 16
num_epochs = 5
learning_rate=0.001

In [ ]:
args = set_train_arguments(
    train_path=train_path,
    batch_size=batch_size,
    num_epochs=num_epochs,
    learning_rate=learning_rate,
)

In [ ]:
trainer = BaseModel(
    model=model,
    hug_model_name=  hug_model_name,
    train_data_iterator = train_data_iterator,
    valid_data_iterator = valid_data_iterator,
    seq2seq_train_args = args,
)

In [ ]:
# train from scratch
trainer.train()

# train from checkpoint
trainer.train_from_checkpoint(last_check_point=last_check_point)

In [ ]:
trainer.evaluate()

In [ ]:
trainer.model_save(model_path=model_path)
trainer.model_upload()